In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import yfinance as yf
from pandas_datareader import data as pdr

In [ ]:
#read data
sets = pd.read_csv('sets.csv', sep=";")
themes = pd.read_csv('themes.csv')

In [ ]:
#merge data into one dataframe
sets_themes = pd.merge(sets, themes, how='left', left_on='theme_id', right_on='id', suffixes=('_sets', '_themes'))
sets_themes = sets_themes.drop(['id', 'parent_id'], axis='columns')